In [5]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [4]:
curs = conn.cursor()

In [8]:
rs = curs.execute("SELECT * from Facilities").fetchall()

In [26]:
rs

In [10]:
df1= pd.DataFrame(rs)

In [11]:
df1

,0,1,2,3,4,5
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [7]:
con.close()

In [16]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do. */
df1 =pd.read_sql("SELECT DISTINCT CASE WHEN name LIKE '% 1' THEN REPLACE( name, ' 1', '' ) WHEN name LIKE '% 2' THEN REPLACE( name, ' 2', '' ) ELSE name END AS name FROM `Facilities`",conn)

In [24]:
df1

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


In [18]:
# /* Q2: How many facilities do not charge a fee to members? */
df1 =pd.read_sql("SELECT facid, name FROM `Facilities` WHERE membercost =0",conn)

In [25]:
df1

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


In [ ]:
# /* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question. */

In [20]:
df1 =pd.read_sql("SELECT `facid` , `name` , `membercost` ,`monthlymaintenance` FROM `Facilities` WHERE membercost < monthlymaintenance * 0.2",conn)

In [26]:
df1

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

In [22]:
df1 =pd.read_sql("SELECT * FROM `Facilities` WHERE facid IN ( 1, 5 )",conn)

In [27]:
df1

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

In [28]:
df1 =pd.read_sql("SELECT facid, name, CASE WHEN `monthlymaintenance` >100 THEN 'Expensive' ELSE 'Cheap' END AS fac_label FROM `Facilities` GROUP BY 1 , 2, 3",conn)

In [29]:
df1

,facid,name,fac_label
0,0,Tennis Court 1,Expensive
1,1,Tennis Court 2,Expensive
2,2,Badminton Court,Cheap
3,3,Table Tennis,Cheap
4,4,Massage Room 1,Expensive
5,5,Massage Room 2,Expensive
6,6,Squash Court,Cheap
7,7,Snooker Table,Cheap
8,8,Pool Table,Cheap


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

In [31]:
df1 =pd.read_sql("select firstname,surname FROM (SELECT firstname,surname,memid FROM `Members` WHERE joindate = (select MAX(joindate) from `Members`)) ",conn)

In [32]:
df1

,firstname,surname
0,Darren,Smith


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

In [37]:
df1 =pd.read_sql("SELECT m.firstname+ ' '+ m.surname as name, f.name FROM `Bookings` b JOIN ( SELECT facid, name FROM `Facilities` WHERE name LIKE 'Tennis Court%' ) f ON b.facid = f.facid JOIN `Members` m ON b.memid = m.memid GROUP BY 1 , 2 ORDER BY 1 , 2",conn)                

In [38]:
df1

,name,name
0,0,Tennis Court 1
1,0,Tennis Court 2


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

In [40]:
df1 =pd.read_sql(""" SELECT CASE WHEN  m.firstname+ ' '+ m.surname  LIKE 'GUEST%'
THEN 'GUEST'
ELSE m.firstname+ ' '+ m.surname 
END AS name, f.facid, f.name AS fac_name
FROM `Bookings` b
JOIN `Facilities` f ON b.facid = f.facid
JOIN `Members` m ON b.memid = m.memid
WHERE CAST( b.starttime AS DATE ) = CAST( '2012-09-14' AS DATE )
AND (
f.membercost >30
OR f.guestcost >30
)
GROUP BY 1 , 2, 3""",conn)

In [41]:
df1

,name,facid,fac_name
0,0,4,Massage Room 1
1,0,5,Massage Room 2


/* Q9: This time, produce the same result as in Q8, but using a subquery. */

In [43]:
df1 =pd.read_sql("""SELECT CASE WHEN  m.firstname + ' ' + m.surname  LIKE 'GUEST%'
THEN 'GUEST'
ELSE  m.firstname +' ' + m.surname 
END AS name, f.facid, f.name AS fac_name
FROM (

SELECT *
FROM `Bookings`
WHERE CAST( starttime AS DATE ) = CAST( '2012-09-14' AS DATE )
)b
JOIN (

SELECT *
FROM `Facilities`
WHERE (
membercost >30
OR guestcost >30
)
)f ON b.facid = f.facid
JOIN `Members` m ON b.memid = m.memid
GROUP BY 1 , 2, 3 """,conn)

In [44]:
df1

,name,facid,fac_name
0,0,4,Massage Room 1
1,0,5,Massage Room 2


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [45]:
df1 =pd.read_sql(""" SELECT b.facid, SUM( (
mems * membercost
) + ( guests * guestcost ) ) AS total_revenue
FROM (
SELECT facid, COUNT( DISTINCT
CASE WHEN memid <>0
THEN memid
END ) mems, COUNT( DISTINCT
CASE WHEN memid =0
THEN memid
END ) guests
FROM `Bookings`
GROUP BY 1
)b
JOIN `Facilities` f ON b.facid = f.facid
GROUP BY 1
HAVING total_revenue <1000
ORDER BY 2""",conn)

In [46]:
df1

,facid,total_revenue
0,3,5.0
1,7,5.0
2,8,5.0
3,2,15.5
4,6,101.5
5,1,130.0
6,0,140.0
7,5,198.8
8,4,317.6


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [48]:
df1 =pd.read_sql("""SELECT m1.firstname, m2.surname, m1.recommendedby, m2.firstname+ ' '+ m2.surname AS recommendedby
FROM Members m1
JOIN Members m2 ON m1.recommendedby = m2.memid
WHERE m2.firstname NOT LIKE '%GUEST%'
GROUP BY 1 , 2, 3, 4
ORDER BY 2 , 1 """,conn)

In [49]:
df1

,firstname,surname,recommendedby,recommendedby
0,Ramnaresh,Bader,15,0
1,Joan,Baker,16,0
2,Matthew,Butters,5,0
3,David,Farrell,13,0
4,Timothy,Farrell,13,0
5,Henrietta,Genting,20,0
6,Douglas,Jones,11,0
7,David,Joplette,4,0
8,Nancy,Joplette,4,0
9,John,Purview,30,0


/* Q12: Find the facilities with their usage by member, but not guests */

In [50]:
df1 =pd.read_sql("""SELECT b.facid, m.firstname, b.usages
FROM (
SELECT facid, memid, COUNT( * ) AS usages
FROM Bookings
WHERE memid <>0
GROUP BY 1 , 2
)b
JOIN Members m ON b.memid = m.memid """,conn)

In [51]:
df1

,facid,firstname,usages
0,0,Tracy,30
1,0,Tim,6
2,0,Janice,19
3,0,Gerald,57
4,0,Burton,31
...,...,...,...
197,8,Henrietta,3
198,8,David,25
199,8,Henry,33
200,8,Millicent,5


/* Q13: Find the facilities usage by month, but not guests */

In [52]:

df1 =pd.read_sql("""select b.facid, f.name, b.usage_month,b.usages
from
(SELECT facid, SUBSTR( `starttime` , 1, 7 ) AS usage_month, COUNT( * ) AS usages
FROM Bookings
WHERE memid <>0
GROUP BY 1 , 2
) b
join Facilities f ON b.facid = f.facid
order by 1,3 """,conn)

In [53]:
df1

,facid,name,usage_month,usages
0,0,Tennis Court 1,2012-07,65
1,0,Tennis Court 1,2012-08,111
2,0,Tennis Court 1,2012-09,132
3,1,Tennis Court 2,2012-07,41
4,1,Tennis Court 2,2012-08,109
5,1,Tennis Court 2,2012-09,126
6,2,Badminton Court,2012-07,51
7,2,Badminton Court,2012-08,132
8,2,Badminton Court,2012-09,161
9,3,Table Tennis,2012-07,48
